In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

2024-03-24 14:04:20.470352: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-24 14:04:20.736652: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 14:04:20.736676: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 14:04:20.737851: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 14:04:20.847465: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-03-24 14:04:23.434341: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 14:04:23.541050: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 14:04:23.541332: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
train_data_path = "BIRDS1_split/train"
test_data_path = "BIRDS1_split/test"
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
bird_classes = sorted(os.listdir(train_data_path))

In [3]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)


Found 25744 files belonging to 167 classes.
Using 18021 files for training.


2024-03-24 19:04:29.692564: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 19:04:29.692915: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 19:04:29.693073: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 25744 files belonging to 167 classes.
Using 7723 files for validation.
Found 11545 files belonging to 167 classes.


In [4]:
preprocess_input = tf.keras.applications.nasnet.preprocess_input
train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_dataset = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [13]:
base_model = tf.keras.applications.NASNetMobile(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)
base_model.trainable = False

for layer in base_model.layers:
    print(layer.name)

input_3
stem_conv1
stem_bn1
activation_188
reduction_conv_1_stem_1
reduction_bn_1_stem_1
activation_189
activation_191
separable_conv_1_pad_reduction_left1_stem_1
separable_conv_1_pad_reduction_right1_stem_1
separable_conv_1_reduction_left1_stem_1
separable_conv_1_reduction_right1_stem_1
separable_conv_1_bn_reduction_left1_stem_1
separable_conv_1_bn_reduction_right1_stem_1
activation_190
activation_192
separable_conv_2_reduction_left1_stem_1
separable_conv_2_reduction_right1_stem_1
activation_193
separable_conv_2_bn_reduction_left1_stem_1
separable_conv_2_bn_reduction_right1_stem_1
separable_conv_1_pad_reduction_right2_stem_1
activation_195
reduction_add_1_stem_1
separable_conv_1_reduction_right2_stem_1
separable_conv_1_pad_reduction_right3_stem_1
activation_197
separable_conv_1_bn_reduction_right2_stem_1
separable_conv_1_reduction_right3_stem_1
separable_conv_1_reduction_left4_stem_1
activation_194
separable_conv_1_bn_reduction_right3_stem_1
separable_conv_1_bn_reduction_left4_stem_1


In [14]:
from keras.src import regularizers

inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3), name="input_layer")

x = base_model(inputs, training=False)
x = layers.GlobalAvgPool2D()(x)

x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)

x = tf.keras.layers.Dropout(0.5)(x)

outputs = layers.Dense(len(bird_classes), activation='softmax', dtype=tf.float32, kernel_regularizer=regularizers.l2(0.004))(x)

model1 = tf.keras.Model(inputs, outputs)

In [15]:
model1.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 global_average_pooling2d_2  (None, 1056)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_16 (Dense)            (None, 512)               541184    
                                                                 
 dense_17 (Dense)            (None, 512)               262656    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                           

In [16]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=3) 
len(base_model.layers)

769

In [17]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='Best_models/best_nasnet/', verbose=1, save_best_only=True)

model1.fit(train_dataset,
                                        validation_data=validation_dataset,

                                        epochs=10,
                                        verbose=1,
                                        callbacks=[checkpointer]) 

Epoch 1/10


2024-03-24 19:05:54.349855: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-03-24 19:05:54.591931: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-24 19:05:55.689229: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x785d1400a4b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-24 19:05:55.689269: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-03-24 19:05:55.698657: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-24 19:05:55.792939: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


564/564 [==============================] - ETA: 0s - loss: 4.8326 - accuracy: 0.0333
Epoch 1: val_loss improved from inf to 3.83091, saving model to Best_models/best_nasnet/
564/564 [==============================] - 64s 100ms/step - loss: 4.8326 - accuracy: 0.0333 - val_loss: 3.8309 - val_accuracy: 0.1518
Epoch 2/10
563/564 [============================>.] - ETA: 0s - loss: 3.8255 - accuracy: 0.0871
Epoch 2: val_loss improved from 3.83091 to 3.24719, saving model to Best_models/best_nasnet/
564/564 [==============================] - 57s 100ms/step - loss: 3.8253 - accuracy: 0.0872 - val_loss: 3.2472 - val_accuracy: 0.1747
Epoch 3/10
563/564 [============================>.] - ETA: 0s - loss: 3.4000 - accuracy: 0.1299
Epoch 3: val_loss improved from 3.24719 to 2.98890, saving model to Best_models/best_nasnet/
564/564 [==============================] - 60s 107ms/step - loss: 3.4003 - accuracy: 0.1299 - val_loss: 2.9889 - val_accuracy: 0.1941
Epoch 4/10
564/564 [==========================

In [19]:
for layer in base_model.layers[-100:]:
    layer.trainable = True


In [20]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])
start_epoch = 10

model1.fit(train_dataset,
                                        validation_data=validation_dataset,
                                        epochs=start_epoch+10,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 11/20
564/564 [==============================] - ETA: 0s - loss: 2.3640 - accuracy: 0.3251
Epoch 11: val_loss improved from 1.97917 to 1.90133, saving model to Best_models/best_nasnet/
564/564 [==============================] - 65s 99ms/step - loss: 2.3640 - accuracy: 0.3251 - val_loss: 1.9013 - val_accuracy: 0.4322 - lr: 0.0010
Epoch 12/20
564/564 [==============================] - ETA: 0s - loss: 2.2809 - accuracy: 0.3415
Epoch 12: val_loss improved from 1.90133 to 1.82014, saving model to Best_models/best_nasnet/
564/564 [==============================] - 58s 102ms/step - loss: 2.2809 - accuracy: 0.3415 - val_loss: 1.8201 - val_accuracy: 0.4725 - lr: 0.0010
Epoch 13/20
563/564 [============================>.] - ETA: 0s - loss: 2.2071 - accuracy: 0.3610
Epoch 13: val_loss improved from 1.82014 to 1.68646, saving model to Best_models/best_nasnet/
564/564 [==============================] - 60s 106ms/step - loss: 2.2074 - accuracy: 0.3611 - val_loss: 1.6865 - val_accuracy: 0.5137 

In [21]:

for layer in base_model.layers[-200:]:
    layer.trainable = True


In [22]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(0.0005),
                           metrics=['accuracy'])
start_epoch = 20
model1.fit(train_dataset,
                                        
                                        validation_data=validation_dataset,
                                        
                                        epochs=start_epoch+10,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer]) 

Epoch 21/30
564/564 [==============================] - ETA: 0s - loss: 1.6590 - accuracy: 0.4986
Epoch 21: val_loss improved from 1.44643 to 1.27467, saving model to Best_models/best_nasnet/
564/564 [==============================] - 66s 104ms/step - loss: 1.6590 - accuracy: 0.4986 - val_loss: 1.2747 - val_accuracy: 0.6171
Epoch 22/30
563/564 [============================>.] - ETA: 0s - loss: 1.5819 - accuracy: 0.5170
Epoch 22: val_loss improved from 1.27467 to 1.24272, saving model to Best_models/best_nasnet/
564/564 [==============================] - 57s 101ms/step - loss: 1.5818 - accuracy: 0.5170 - val_loss: 1.2427 - val_accuracy: 0.6251
Epoch 23/30
564/564 [==============================] - ETA: 0s - loss: 1.5364 - accuracy: 0.5270
Epoch 23: val_loss improved from 1.24272 to 1.23374, saving model to Best_models/best_nasnet/
564/564 [==============================] - 57s 101ms/step - loss: 1.5364 - accuracy: 0.5270 - val_loss: 1.2337 - val_accuracy: 0.6109
Epoch 24/30
563/564 [====

In [23]:

for layer in base_model.layers[-300:]:
    layer.trainable = True


In [24]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(learning_rate=0.0003),
                           metrics=['accuracy'])
start_epoch = 30

model1.fit(train_dataset,
                                        validation_data=validation_dataset,
                                       
                                        epochs=start_epoch+20,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 31/50
563/564 [============================>.] - ETA: 0s - loss: 1.2710 - accuracy: 0.6011
Epoch 31: val_loss improved from 1.11567 to 1.01905, saving model to Best_models/best_nasnet/
564/564 [==============================] - 65s 103ms/step - loss: 1.2710 - accuracy: 0.6011 - val_loss: 1.0190 - val_accuracy: 0.6896 - lr: 3.0000e-04
Epoch 32/50
563/564 [============================>.] - ETA: 0s - loss: 1.2287 - accuracy: 0.6135
Epoch 32: val_loss did not improve from 1.01905
564/564 [==============================] - 28s 50ms/step - loss: 1.2287 - accuracy: 0.6135 - val_loss: 1.0211 - val_accuracy: 0.7135 - lr: 3.0000e-04
Epoch 33/50
563/564 [============================>.] - ETA: 0s - loss: 1.2120 - accuracy: 0.6176
Epoch 33: val_loss improved from 1.01905 to 1.01899, saving model to Best_models/best_nasnet/
564/564 [==============================] - 58s 102ms/step - loss: 1.2121 - accuracy: 0.6176 - val_loss: 1.0190 - val_accuracy: 0.7022 - lr: 3.0000e-04
Epoch 34/50
562/564 [

In [25]:

for layer in base_model.layers[-400:]:
    layer.trainable = True


In [26]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(learning_rate=0.0001),
                           metrics=['accuracy'])
start_epoch = 50

model1.fit(train_dataset,
                                        validation_data=validation_dataset,
                                       
                                        epochs=start_epoch+10,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 51/60
563/564 [============================>.] - ETA: 0s - loss: 0.9589 - accuracy: 0.6906
Epoch 51: val_loss improved from 0.89066 to 0.84903, saving model to Best_models/best_nasnet/
564/564 [==============================] - 66s 105ms/step - loss: 0.9588 - accuracy: 0.6907 - val_loss: 0.8490 - val_accuracy: 0.7799 - lr: 1.0000e-04
Epoch 52/60
563/564 [============================>.] - ETA: 0s - loss: 0.9406 - accuracy: 0.6915
Epoch 52: val_loss did not improve from 0.84903
564/564 [==============================] - 28s 49ms/step - loss: 0.9405 - accuracy: 0.6915 - val_loss: 0.8575 - val_accuracy: 0.7712 - lr: 1.0000e-04
Epoch 53/60
563/564 [============================>.] - ETA: 0s - loss: 0.9327 - accuracy: 0.6946
Epoch 53: val_loss did not improve from 0.84903
564/564 [==============================] - 29s 52ms/step - loss: 0.9326 - accuracy: 0.6947 - val_loss: 0.8654 - val_accuracy: 0.7404 - lr: 1.0000e-04
Epoch 54/60
563/564 [============================>.] - ETA: 0s - los

In [25]:

for layer in base_model.layers[-550:]:
    layer.trainable = True


In [27]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(learning_rate=0.00004),
                           metrics=['accuracy'])
start_epoch = 60

model1.fit(train_dataset,
                                        validation_data=validation_dataset,
                                       
                                        epochs=start_epoch+10,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 61/70
564/564 [==============================] - ETA: 0s - loss: 0.8789 - accuracy: 0.7154
Epoch 61: val_loss improved from 0.83709 to 0.83432, saving model to Best_models/best_nasnet/
564/564 [==============================] - 67s 107ms/step - loss: 0.8789 - accuracy: 0.7154 - val_loss: 0.8343 - val_accuracy: 0.8077 - lr: 4.0000e-05
Epoch 62/70
562/564 [============================>.] - ETA: 0s - loss: 0.8745 - accuracy: 0.7143
Epoch 62: val_loss improved from 0.83432 to 0.83183, saving model to Best_models/best_nasnet/
564/564 [==============================] - 55s 97ms/step - loss: 0.8746 - accuracy: 0.7142 - val_loss: 0.8318 - val_accuracy: 0.7944 - lr: 4.0000e-05
Epoch 63/70
563/564 [============================>.] - ETA: 0s - loss: 0.8722 - accuracy: 0.7169
Epoch 63: val_loss did not improve from 0.83183
564/564 [==============================] - 28s 49ms/step - loss: 0.8721 - accuracy: 0.7169 - val_loss: 0.8352 - val_accuracy: 0.8099 - lr: 4.0000e-05
Epoch 64/70
563/564 [=

In [28]:

for layer in base_model.layers[-750:]:
    layer.trainable = True


In [32]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(learning_rate=0.00001),
                           metrics=['accuracy'])
start_epoch = 80

model1.fit(train_dataset,
                                        validation_data=validation_dataset,
                                       
                                        epochs=start_epoch+10,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 81/90
563/564 [============================>.] - ETA: 0s - loss: 1.3236 - accuracy: 0.6117
Epoch 81: val_loss did not improve from 0.80952
564/564 [==============================] - 37s 56ms/step - loss: 1.3239 - accuracy: 0.6117 - val_loss: 0.9863 - val_accuracy: 0.7285 - lr: 1.0000e-05
Epoch 82/90
564/564 [==============================] - ETA: 0s - loss: 1.2248 - accuracy: 0.6371
Epoch 82: val_loss did not improve from 0.80952
564/564 [==============================] - 29s 51ms/step - loss: 1.2248 - accuracy: 0.6371 - val_loss: 0.9308 - val_accuracy: 0.7457 - lr: 1.0000e-05
Epoch 83/90
516/564 [==========================>...] - ETA: 1s - loss: 1.1683 - accuracy: 0.6536

KeyboardInterrupt: 

In [13]:
import tensorflow as tf
test_dataset_224 = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(224, 224),
    shuffle=False,
    seed=123,

)
test_dataset_299 = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(299, 299),
    shuffle=False,
    seed=123,

)
preprocess_input = tf.keras.applications.mobilenet_v3.preprocess_input

test_dataset_224 = test_dataset_224.map(lambda x, y: (preprocess_input(x), y))
test_dataset_299 = test_dataset_299.map(lambda x, y: (preprocess_input(x), y))

2024-03-24 14:46:44.536428: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-24 14:46:44.564019: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 14:46:44.564090: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 14:46:44.564134: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 14:46:44.570442: I tensorflow/core/platform/cpu_feature_g

Found 11545 files belonging to 167 classes.


2024-03-24 14:46:46.559885: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 14:46:46.588070: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 14:46:46.588288: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 11545 files belonging to 167 classes.


In [16]:
y_labels_224 = []
y_labels_299 = []
for images, labels in test_dataset_224.unbatch():
    y_labels_224.append(labels.numpy().argmax())
for images, labels in test_dataset_299.unbatch():
    y_labels_299.append(labels.numpy().argmax())

In [ ]:
def calculate_accuracy_top3(row):
    if row['True Label'] in row['top 3 cols']:
        return 1
    else:
        return 0

In [28]:
import pandas as pd

eff_b7 = tf.keras.models.load_model("Best_models/best_eff_b7_90.67")
preds_eff_b7 = pd.DataFrame(eff_b7.predict(test_dataset_224))
top3_labels = utils.add_top_three_row_values(preds_eff_b7, y_labels_224)
top3_labels['Accuracy_Top3'] = top3_labels.apply(calculate_accuracy_top3, axis=1)
overall_accuracy_top3 = top3_labels['Accuracy_Top3'].mean()
overall_accuracy_top3

361/361 [==============================] - 52s 136ms/step


0.9887397141619749

In [31]:
eff_v2L = tf.keras.models.load_model("Best_models/best_effnetv2L_90.57")

preds_eff_v2L = pd.DataFrame(eff_v2L.predict(test_dataset_224))

top3_labels_effv2L = utils.add_top_three_row_values(preds_eff_v2L, y_labels_224)

top3_labels_effv2L['Accuracy_Top3'] = top3_labels_effv2L.apply(calculate_accuracy_top3, axis=1)

overall_accuracy_top3_effv2L = top3_labels_effv2L['Accuracy_Top3'].mean()

overall_accuracy_top3_effv2L

361/361 [==============================] - 42s 107ms/step


0.9799913382416631

In [32]:
mobilenet_vsmall = tf.keras.models.load_model("Best_models/best_mobilenet_small_89.65")

preds_mobilenet_vsmall= pd.DataFrame(mobilenet_vsmall.predict(test_dataset_224))

top3_mobilenet_vsmall  = utils.add_top_three_row_values(preds_mobilenet_vsmall, y_labels_224)

top3_mobilenet_vsmall['Accuracy_Top3'] = top3_mobilenet_vsmall.apply(calculate_accuracy_top3, axis=1)

overall_accuracy_top3_mobilenet_vsmall= top3_mobilenet_vsmall['Accuracy_Top3'].mean()

overall_accuracy_top3_mobilenet_vsmall

361/361 [==============================] - 5s 12ms/step


0.9711563447379818

In [34]:
mobilenet_vlarge = tf.keras.models.load_model("Best_models/best_mobilenetv3large_90.67")

preds_mobilenet_vlarge = pd.DataFrame(mobilenet_vlarge.predict(test_dataset_224))

top3_mobilenet_vlarge  = utils.add_top_three_row_values(preds_mobilenet_vlarge, y_labels_224)

top3_mobilenet_vlarge['Accuracy_Top3'] = top3_mobilenet_vlarge.apply(calculate_accuracy_top3, axis=1)

overall_accuracy_top3_mobilenet_vlarge= top3_mobilenet_vlarge['Accuracy_Top3'].mean()

overall_accuracy_top3_mobilenet_vlarge

361/361 [==============================] - 7s 16ms/step


0.9778258986574274